In [34]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [35]:
pip install pandas

In [40]:
 import pandas as pd
import requests
import io

# Define the URL of the XLSX file
xlsx_url = 'https://docs.google.com/spreadsheets/d/1mmKTye9dCq15F1SD1pzYwH-CPDW_5jLWY9rnM8vM-1I/gviz/tq?tqx=out:csv'

# Send a GET request to fetch the XLSX data
response = requests.get(xlsx_url)

if response.status_code == 200:
    # Read the XLSX data into a DataFrame
    df = pd.read_csv(io.StringIO(response.text))

    # Now you can work with the 'df' DataFrame as needed.
    print(df.head())
else:
    print(f"Failed to retrieve the XLSX file. Status code: {response.status_code}")


  Position ID Position Status                 Time            Time Out  \
0   WFS000054          Active                  NaN                 NaN   
1   WFS000065          Active  09/12/2023 10:08 AM  09/12/2023 1:53 PM   
2   WFS000065          Active   09/12/2023 2:23 PM  09/12/2023 7:02 PM   
3   WFS000065          Active  09/13/2023 10:08 AM  09/13/2023 2:20 PM   
4   WFS000065          Active   09/13/2023 2:50 PM  09/13/2023 8:44 PM   

  Timecard Hours (as Time) Pay Cycle Start Date Pay Cycle End Date  \
0                      NaN                  NaN                NaN   
1                     3:45           09/10/2023         09/23/2023   
2                     4:39           09/10/2023         09/23/2023   
3                     4:12           09/10/2023         09/23/2023   
4                     5:54           09/10/2023         09/23/2023   

     Employee Name  File Number  
0  SiWgh, PraGhjEM           54  
1  REsaXiaWE, XAis           65  
2  REsaXiaWE, XAis           65 

In [44]:
import pandas as pd

# Read the Google Sheets data into a DataFrame (replace 'your_data.csv' with your actual file path)
data = pd.read_csv('https://docs.google.com/spreadsheets/d/1mmKTye9dCq15F1SD1pzYwH-CPDW_5jLWY9rnM8vM-1I/gviz/tq?tqx=out:csv')

# Ensure that the 'Time' column is in datetime format
data['Time'] = pd.to_datetime(data['Time'])
data['Time Out'] = pd.to_datetime(data['Time Out'])

# Sort the DataFrame by 'Employee Name' and 'Time' for better analysis
data.sort_values(['Employee Name', 'Time'], inplace=True)

# Initialize counters for each condition
consecutive_workdays_count = 0
less_than_10_hours_count = 0
more_than_14_hours_count = 0

# Initialize variables to track consecutive workdays and previous record for each employee
consecutive_workdays = 0
prev_record = None

# Iterate over the records in the DataFrame
for index, row in data.iterrows():
    # Check if the employee name has changed
    if prev_record is None or prev_record['Employee Name'] != row['Employee Name']:
        consecutive_workdays = 0
    else:
        # Calculate the time difference between consecutive shifts for the same employee
        time_diff_hours = (row['Time'] - prev_record['Time Out']).total_seconds() / 3600

        # Condition a) Employees who have worked for 7 consecutive days
        if consecutive_workdays >= 6:
            consecutive_workdays_count += 1

        # Condition b) Employees with less than 10 hours between shifts but more than 1 hour
        if 1 < time_diff_hours < 10:
            less_than_10_hours_count += 1

        # Condition c) Employees who have worked for more than 14 hours in a single shift
        if row['Timecard Hours (as Time)'] > '14:00':
            more_than_14_hours_count += 1

    prev_record = row
    consecutive_workdays += 1

# Print the counts for each condition
print(f"Employees who have worked for 7 consecutive days: {consecutive_workdays_count}")
print(f"Employees with less than 10 hours between shifts but more than 1 hour: {less_than_10_hours_count}")
print(f"Employees who have worked for more than 14 hours in a single shift: {more_than_14_hours_count}")


Employees who have worked for 7 consecutive days: 907
Employees with less than 10 hours between shifts but more than 1 hour: 11
Employees who have worked for more than 14 hours in a single shift: 1352
